Group 9 Project 
Riley Fritz, Sofanit Habte, Robert Boulka
Loading transformed Data from 2019 confirmed fires in the city of Minneapolis

In [1]:
#importing dependencies
import pandas as pd
from sqlalchemy import create_engine
from config import password

In [2]:
#making connection to SQL database
rds_connection_string = "postgres:{password}@localhost:5432/fire_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [3]:
conn = engine.connect()
#session = Session(bind=engine)

Extracting and Transforming the fires confirmed data

In [5]:
#load csv
file = 'Resources/Fires_Confirmed_2019.csv'
df1 = pd.read_csv(file)

In [6]:
#organizing data
df2 = df1.loc[:,["inci_no", "alm_date", "number", "street", "st_type", "st_suffix", "latitude", "longitude", "inci_type"]]

In [7]:
#transforming and joining street address columns
df2['street_address'] = df2['street'] + " " + df2['st_type'] + " " + df2['st_suffix']

In [8]:
#dropping the "19-" from incident number column
df2['inci_no'] = df2['inci_no'].str[3:]

In [9]:
#dropping timestamp from datetime column
df2['alm_date'] = pd.to_datetime(df2['alm_date'])
df2['date'] = df2['alm_date'].dt.date

In [10]:
#renaming columns to match database table
df3 = df2.rename(columns = {'inci_no': 'incident_number', 'number': 'street_number', 'inci_type': 'incident_type', 'latitude': 'latitude', 'longitude': 'longitude',})
df3.head()

,incident_number,alm_date,street_number,street,st_type,st_suffix,latitude,longitude,incident_type,street_address,date
0,0000150,2019-01-02,2433,5,AVE,S,44.958567,-93.268979,113,5 AVE S,2019-01-02
1,0000205,2019-01-02,,Cedar,AVE,S,44.966616,-93.247251,131,Cedar AVE S,2019-01-02
2,0000250,2019-01-02,810,Thornton,ST,SE,44.966156,-93.219234,113,Thornton ST SE,2019-01-02
3,0000300,2019-01-03,1325,4,ST,SE,44.980903,-93.236813,251,4 ST SE,2019-01-03
4,0000342,2019-01-03,,7,ST,S,44.966616,-93.247251,151,7 ST S,2019-01-03


In [11]:
#reorganizing columns to match database table
fires_reported_df = df3[['incident_number', 'date', 'incident_type', 'street_number', 'street_address', 'latitude', 'longitude']]
fires_reported_df.head()

,incident_number,date,incident_type,street_number,street_address,latitude,longitude
0,0000150,2019-01-02,113,2433,5 AVE S,44.958567,-93.268979
1,0000205,2019-01-02,131,,Cedar AVE S,44.966616,-93.247251
2,0000250,2019-01-02,113,810,Thornton ST SE,44.966156,-93.219234
3,0000300,2019-01-03,251,1325,4 ST SE,44.980903,-93.236813
4,0000342,2019-01-03,151,,7 ST S,44.966616,-93.247251


In [12]:
#loading dataframe into database table
fires_reported_df.to_sql(name='fires_confirmed', con=engine, if_exists='append', index=False)